# Manipulating Norm Vectors

We use Doc2Vec to create sentence representations over contracts.
Using such representations, we get a random norm sentence representation and modal verb (shall, may, must, etc) representations.
We subtract the norm representation by the modal verb representation in the norm.
Then, we add a modal verb representation to the norm with an oposite deontic meaning (e.g., remove a prohibition and add a permission).
As result, we expect to obtain potential conflicts by 

## Doc2Vec

We train a doc2vec model using norm sentences from 169 manufacturing contracts.


In [1]:
import sys
import pickle
import numpy as np
import process_norms
from process_norms import ProcessNorms

Using TensorFlow backend.
/home/aires/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
d2v_path = 'models/model_2017-11-03_21-39-30.doc2vec'
dict_path = 'models/model_2017-11-03_21-39-30.pkl'
sent_cls_path = 'sentence_classifier/classifiers/17-11-03_18:45/sentence_classifier_17-11-03_18:45.pkl'
sent_cls_names_path = 'sentence_classifier/classifiers/17-11-03_18:45/sentence_classifier_dict_17-11-03_18:45.pkl'

In [3]:
def load_dict(path):

    dict_file = open(path, 'r')
    p_dict = pickle.load(dict_file)

    return p_dict

In [4]:
def print_list(l):
    
    return ' '.join(l)

In [5]:
def print_similar(similar_vecs):
# Run over similar vectors and print the sentences.
    for similar in similar_vecs:
        print '{}\t {}\n'.format(print_list(sent_dict[similar[0]]), similar[1])

In [6]:
# Instantiate the ProcessNorms class.
pn = ProcessNorms(d2v_path, sent_cls_path, sent_cls_names_path)

In [7]:
# Load dict.
sent_dict = load_dict(dict_path)

In [21]:
# Get a random norm sentence.
random_key = 'SENT_777'
if random_key in pn.doc2vec_model.docvecs:
    rand_sent_vec = pn.doc2vec_model.docvecs[random_key]
    print print_list(sent_dict[random_key])
else:
    print "Unknown key."

If a shipment of Finished Product ordered by Santarus under this Agreement has not been delivered at the shipping point within *** Business Days after the scheduled delivery date (any date of delivery that is more than *** Business Days after the scheduled delivery date is hereinafter referred to as a "LATE DELIVERY DATE") stated on the corresponding Firm Purchase Order (a "LATE SHIPMENT"), or if the shipment received by Santarus contains less than the quantity specified in the corresponding Firm Purchase Order (a "SHORT SHIPMENT"), Santarus shall notify Patheon promptly upon such discovery and , in any event, not later than *** days after receipt of, or failure to receive, such ordered Finished Product.


In [9]:
# Subtract the obligation word from the sentence.
sub_vector = pn.doc2vec_model['will']  # Get subtraction vector.
add_vector = pn.doc2vec_model['may']  # Get adition vector.
no_modal = np.subtract(rand_sent_vec, sub_vector)  # Subtract vector from our sentence representation.
new_sent = np.add(no_modal, add_vector)  # Add vector to the sentence representation.

In [10]:
# Find similar vectors based on our new vector.
similar_vecs = pn.doc2vec_model.docvecs.most_similar([new_sent], topn=3)
print_similar(similar_vecs)

The indemnitee shall be entitled to any and all remedies available at law or in equity, including without limitation, damages and all equitable remedies, as a means of collecting the indemnification to which entitled.	 0.367222994566

Notwithstanding the provision of this Section 13.2, the Parties agree that certain violations or threatened violations of this Agreement will result in irrevocable harm to other Party, for which damages would be an inadequate remedy.	 0.362473934889

Topps shall then, at its sole expense, (i) settle or defend (with counsel of its own choice) any such claim brought against Hershey and/or Topps, (ii) procure for Topps the right or rights necessary to manufacture, package and sell the Packaged Product or replace or modify the Product, Packaging or Packaged Product, and (iii) to the extent required by a court adjudicating the claim or as agreed in a settlement agreement respecting the claim, remove the affected Product, Packaging and Packaged Product from Her

In [11]:
# Find similar vectors based on the original sentence.
similar_vecs = pn.doc2vec_model.docvecs.most_similar(random_key, topn=3)
print_similar(similar_vecs)

In any action to enforce this Agreement the prevailing party will be entitled to costs and attorneys' fees.	 0.936549544334

In any dispute resolution proceeding between the parties in connection with this Agreement, the prevailing party will be entitled to its reasonable attorney's fees and costs in such proceeding.	 0.653523206711

In any action to enforce this Agreement, the prevailing party shall be awarded all court costs and reasonable attorney fees incurred.	 0.613260388374

